In [ ]:
from flask import Flask
from flask import request
from flask import render_template
import numpy as np
import cv2
import re
from textblob import TextBlob
from imutils.object_detection import non_max_suppression
import pytesseract
from matplotlib import pyplot as plt

app = Flask(__name__)
@app.route('/')
def my_form():
    return render_template("my_form.html")
@app.route('/', methods=['POST'])
def my_form_post():
    st=[]
    data1 = request.form['file']
    print(data1)
    
    imagepath = "C://Users//Abhilasha//Desktop//"+data1
    print(imagepath)

    image = cv2.imread(imagepath)


    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    orig = gray.copy()
    (origH, origW) = image.shape[:2]

    (newW, newH) = (640,640)

    rW = origW / float(newW)
    rH = origH / float(newH)

    image = cv2.resize(image, (newW, newH))
    (H, W) = image.shape[:2]

    blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
        (123.68, 116.78, 103.94), swapRB=True, crop=False)

    net = cv2.dnn.readNet(r'C:\\Users\\Abhilasha\\final_hackabit1\\frozen_east_text_detection.pb')

 
    layerNames = [
        "feature_fusion/Conv_7/Sigmoid",
        "feature_fusion/concat_3"]
    net.setInput(blob)
    (scores, geometry) = net.forward(layerNames)
    def predictions(prob_score, geo):
        (numR, numC) = prob_score.shape[2:4]
        boxes = []
        confidence_val = []

        for y in range(0, numR):
            scoresData = prob_score[0, 0, y]
            x0 = geo[0, 0, y]
            x1 = geo[0, 1, y]
            x2 = geo[0, 2, y]
            x3 = geo[0, 3, y]
            anglesData = geo[0, 4, y]

            for i in range(0, numC):
                if scoresData[i] < 0.5:
                    continue

                (offX, offY) = (i * 4.0, y * 4.0)

                angle = anglesData[i]
                cos = np.cos(angle)
                sin = np.sin(angle)

                h = x0[i] + x2[i]
                w = x1[i] + x3[i]

                endX = int(offX + (cos * x1[i]) + (sin * x2[i]))
                endY = int(offY - (sin * x1[i]) + (cos * x2[i]))
                startX = int(endX - w)
                startY = int(endY - h)

                boxes.append((startX, startY, endX, endY))
                confidence_val.append(scoresData[i])

        return (boxes, confidence_val)
    (boxes, confidence_val) = predictions(scores, geometry)
    boxes = non_max_suppression(np.array(boxes), probs=confidence_val)
    results = []
    for (startX, startY, endX, endY) in boxes:
        startX = int(startX * rW)
        startY = int(startY * rH)
        endX = int(endX * rW)
        endY = int(endY * rH)
        r = orig[startY:endY, startX:endX]
        configuration = ("-l eng --oem 1 --psm 6")
        pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract'
        text = pytesseract.image_to_string(r, config=configuration)
        results.append(((startX, startY, endX, endY), text))

    orig_image = orig.copy()

    for ((start_X, start_Y, end_X, end_Y), text) in results:
        print("{}\n".format(text))
        st.append(format(text))
        text = "".join([x if ord(x) < 128 else "" for x in text]).strip()
    list_1_string = ' '.join(st)
    list_1_string = " ".join(re.findall(r"[a-zA-Z0-9]+", list_1_string))
    list_1_string = TextBlob(list_1_string.lower())
    text = list_1_string.correct()
    print(text)
    return render_template('index.html', message = text)

  
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Oct/2019 08:02:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 08:02:18] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 08:02:19] "GET /favicon.ico HTTP/1.1" 404 -


text1.jpg
C://Users//Abhilasha//Desktop//text1.jpg
Trinity

trinity


127.0.0.1 - - [19/Oct/2019 08:02:36] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [19/Oct/2019 08:02:45] "GET / HTTP/1.1" 200 -


text3.jpg
C://Users//Abhilasha//Desktop//text3.jpg


127.0.0.1 - - [19/Oct/2019 08:03:04] "POST / HTTP/1.1" 200 -


SIRLS

HOSTEL

on

a NEE DOV AR IN! i,

girls hostel on a nee do ar in i
